**Objectives:**


1.   Create BIO tags for available Party One and Party Two data(rolling window of paragraphs)
2.   Preprocessing

**Contents:**


1.   Data Read
2.   Creating of rolling window based text paragraphs(Merge 3 paragraphs in arolling window fashion)
3. Clean the annotations available for Party One and Party Two
4. Create BIO style of NER annotations for each of the rolling window paragraph





In [1]:
import os
os.getcwd()
from google.colab import drive
import os
drive.mount('/content/gdrive')

#IF YOU ARE CLONING FOR THE FIRST TIME in colab please uncomment all the below 5 lines of code. 
#Put git user name and password in appropriate places

root_path = 'gdrive/My Drive/Colab Notebooks/' #change dir to folder where you want to clone
os.chdir('/content/' + root_path)
#!git clone https://github.com/selfishhari/rental_meta_extraction.git
!git branch -r | grep -v '\->' | while read remote; do git branch --track "${remote#origin/}" "$remote"; done



root_path = 'rental_meta_extraction/' #change dir to your project folder's src

os.chdir(root_path)
os.getcwd()

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
fatal: not a git repository (or any parent up to mount point /content)
Stopping at filesystem boundary (GIT_DISCOVERY_ACROSS_FILESYSTEM not set).


'/content/gdrive/My Drive/Colab Notebooks/rental_meta_extraction'

In [2]:
!pip install python-docx

In [0]:
import docx, pandas as pd, sys, os, numpy as np

In [4]:
train_annot = pd.read_csv("data/01_raw/TrainingTestSet.csv")

train_annot.sample(5)

,File Name,Aggrement Value,Aggrement Start Date,Aggrement End Date,Renewal Notice (Days),Party One,Party Two
1,Bangalore Agreement_New,NaN,01.05.2015,28.02.2018,15.0,MR AMAR ARUN WAGH,MR SANDEEP DUTTA
6,RentalAgreement_KKR,8300.0,01.05.2014,31.03.2015,30.0,Amitabh Reddy,SrikanthPai
53,343492954-Rent-Agreement-3E,11500.0,01.05.2016,31.03.2015,30.0,ANAND RAJ,BEENA PURATHAYIL MADHAVAN
22,63057680-Rental-Agreement,450.0,17.08.2010,22.08.2010,NaN,Lagoon Landings LLC,John Public
48,288024755-Rental-Agreement-1,5000.0,21.04.2014,20.09.2014,NaN,S T Nagaveni,H N Madhu


In [0]:
train_docx_folder = "data/01_raw/Training_data"

In [6]:
train_fnames = os.listdir(train_docx_folder)

def clean_train_names(x):

  x = x.replace(".docx", "")

  x = x.replace(".pdf", "")

  return x

clean_tfnames = [clean_train_names(x) for x in train_fnames]

train_annot_available = train_annot.loc[train_annot["File Name"].isin(clean_tfnames), :]

print("Only {} documents are available for training. This is very less".format(train_annot_available.shape[0]))

Only 43 documents are available for training. This is very less


**Rolling window of paragraphs:**

I have basically merged 3 paras in a sliding window fashion. The reason for this is that I noticed, the context arounf party one and party two were  spread across the adjacent paragraphs

In [0]:
def read_docx_contents(filename):
  """
  This function reads the docx file and creates a rolling window based paragraphs  
  """
  
  doc = docx.Document(os.path.join(train_docx_folder, filename))

  all_paragraphs = []

  para_windows = []

  for para in doc.paragraphs:

    text = para.text.strip()

    if text == "":
      continue;

    all_paragraphs.append(text)

    if len(all_paragraphs) < 2:

      para_windows.append(text)

    else:

      para_windows.append(" ".join(all_paragraphs[-3:]))

  filename = filename.replace(".docx", " ")

  filename = filename.replace(".pdf", " ")

  df = pd.DataFrame({"fname":[filename]*len(all_paragraphs), "paras":all_paragraphs, "para_windows":para_windows})

  df["para_windows_proc"] = df["para_windows"].str.lower()

  return df

**BIO annotation:**

I take the raw text(rolling window) and provided annotation for Party1/Party2 and create BIOs tags for each word

Following Tags are created

1.   B-P1 -> Party1 Beginning
2.   I-P1 -> Party1 Continuation
3.   B-P2 -> Party2 Beginning
4.   I-P2 -> Party2 Continuation
5.   O -> All other non annotated words

In [0]:
def annotate(row, text):

  """
  Takes annotations and text, creates BIOs tags for each text
  """

  text = text.replace(".", " ")

  text = text.replace(",", " ")

  p1 = row["Party One"]

  
  
  if pd.isnull(p1.values[0]):
    #print(p1)
    p1 = "nullvaluehere"

  else:
    p1 = p1.str.lower().str.strip().values[0]

  

  p1 = p1.replace(".", " ")

  p1_list = p1.split()

  annotations = []
  
  prev_label = "O"

  for word in text.split():

    word_label = "O"

    counter = 0

    done_flag = False

    for term in p1_list:

      if (len(term) < 2) | (term == "mr"):
          continue

      if word == term:
        #print(word, word_label, p1)
        if counter == 0:

          word_label = "B-P1"

          done_flag = True

        elif prev_label == "B-P1":

          word_label = "I-P1"

          done_flag = True

      counter += 1
      

    if done_flag:
      #print(word, word_label)

      prev_label = word_label

      annotations.append(word_label)
      continue

    p2 = row["Party Two"]

    if pd.isnull(p2.values[0]):
      #print(p2)
      p2 = "nullvaluehere"

    else:
      p2 = p2.str.lower().str.strip().values[0]
    
    

    p2 = p2.replace(".", " ")
  
    p2_list = p2.split()

    counter = 0

    for term in p2_list:
      if len(term) < 2  | (term == "mr"):
        continue
      if word == term:

        if counter == 0:

          word_label = "B-P2"

          #print(word, word_label)

        elif prev_label == "B-P2":

          word_label = "I-P2"

          #print(word, word_label)

      counter += 1

    
    prev_label = word_label
    annotations.append(word_label)

  return annotations

In [0]:
def annotate_all_docs(train_fnames, train_annot_available):

  """
  Run annotations for all files
  """

  all_doc_df = pd.DataFrame(columns = ['fname', 'paras', 'para_windows', 'para_windows_proc', 'annotations'])

  train_annot_available = train_annot_available.reset_index(drop=True)

  

  for rownum in range(len(clean_tfnames)):

    row = train_annot_available.loc[train_annot_available["File Name"] == clean_tfnames[rownum], :]

    df = read_docx_contents(os.path.join(train_fnames[rownum]))

    df["annotations"] = df["para_windows_proc"].apply(lambda x: annotate(row, x))

    #print(df.head())

    all_doc_df = pd.concat([all_doc_df, df])

  return all_doc_df
    

In [12]:
annotated_df = annotate_all_docs(train_fnames, train_annot_available)

annotated_df["annot_count"] = annotated_df["annotations"].apply(lambda x: len(set(x)))

annotated_df["annot_count"].value_counts()

1    1235
3     140
2     110
4      34
5      19
Name: annot_count, dtype: int64

In [18]:
annotated_df.sample(5)

,fname,paras,para_windows,para_windows_proc,annotations,annot_count
13,251798216-Rental-Agreement-Format1,"CROSS,VINAYAKA,LAYOUT,CHINNAPANNAHLLI.MARATHAH...",7. Tenant NAME and Address: VIVEK CHANDRA MALL...,7. tenant name and address: vivek chandra mall...,"[O, O, O, O, O, B-P2, I-P2, O, O, O, O, O, O, ...",3
59,294331674-Rental-Agreement,if the tenant breaks one of the agreement’s term;,HOW TO END A CONTINUING AGREEMENT * A fixed te...,how to end a continuing agreement * a fixed te...,"[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",1
23,95421373-Agreement,PARKING: The TENANT is entitled to park 1 cars...,The following keys handed over to the TENANT o...,the following keys handed over to the tenant o...,"[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",1
28,392810415-RENT-AGREEMENT,That the Municipal Corporation holding Tax and...,That the LESSEE herein shall at its own cost k...,that the lessee herein shall at its own cost k...,"[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",1
1,251798216-Rental-Agreement-Format1,For rental agreement request take only the det...,© “Lesconcierges Help Desk” © For rental agree...,© “lesconcierges help desk” © for rental agree...,"[O, O, O, O, O, O, O, O, O, O, O, O, O, O]",1


In [19]:
!pwd

!mkdir data/02_processed/

/content/gdrive/My Drive/Colab Notebooks/rental_meta_extraction
mkdir: cannot create directory ‘data/02_processed/’: File exists


In [0]:
annotated_df.to_csv("data/02_processed/annotated_df_v1.csv", index=False)

In [0]:
!git status

!git add data/02_processed/annotated_df_v1.csv

On branch master
Your branch is up to date with 'origin/master'.

Untracked files:
  (use "git add <file>..." to include in what will be committed)

	data/02_processed/

nothing added to commit but untracked files present (use "git add" to track)


In [0]:
!git status

!git config --local user.email "narahari.bm@protonmail.com"
!git config --local user.name "Narahari B M"

!git commit -m "added annotated df for party1 and party2"

On branch master
Your branch is up to date with 'origin/master'.

Changes to be committed:
  (use "git reset HEAD <file>..." to unstage)

	new file:   data/02_processed/annotated_df_v1.csv

[master e411e5b] added annotated df for party1 and party2
 1 file changed, 1588 insertions(+)
 create mode 100644 data/02_processed/annotated_df_v1.csv


In [0]:
!git push origin master

Counting objects: 5, done.
Delta compression using up to 2 threads.
Compressing objects: 100% (4/4), done.
Writing objects: 100% (5/5), 145.91 KiB | 3.10 MiB/s, done.
Total 5 (delta 0), reused 0 (delta 0)
To https://github.com/selfishhari/rental_meta_extraction.git
   b04992f..e411e5b  master -> master


In [0]:
annotated_df.to_pickle("data/02_processed/annotated_df_v1.pickle")

**Validation Dataset**

In [0]:
def read_docx_contents_valid(filename):
  """
  This function reads the docx file and creates a rolling window based paragraphs  
  """
  
  doc = docx.Document(os.path.join(valid_docx_folder, filename))

  all_paragraphs = []

  para_windows = []

  for para in doc.paragraphs:

    text = para.text.strip()

    if text == "":
      continue;

    all_paragraphs.append(text)

    if len(all_paragraphs) < 2:

      para_windows.append(text)

    else:

      para_windows.append(" ".join(all_paragraphs[-3:]))

  filename = filename.replace(".docx", " ")

  filename = filename.replace(".pdf", " ")

  df = pd.DataFrame({"fname":[filename]*len(all_paragraphs), "paras":all_paragraphs, "para_windows":para_windows})

  df["para_windows_proc"] = df["para_windows"].str.lower()

  return df

def annotate_all_docs_valid(train_fnames, train_annot_available):

  """
  Run annotations for all files
  """

  all_doc_df = pd.DataFrame(columns = ['fname', 'paras', 'para_windows', 'para_windows_proc', 'annotations'])

  train_annot_available = train_annot_available.reset_index(drop=True)  

  for rownum in range(len(clean_vfnames)):

    row = train_annot_available.loc[train_annot_available["File Name"] == clean_vfnames[rownum], :]

    df = read_docx_contents_valid(os.path.join(train_fnames[rownum]))

    df["annotations"] = df["para_windows_proc"].apply(lambda x: annotate(row, x))

    #print(df.head())

    all_doc_df = pd.concat([all_doc_df, df])

  return all_doc_df
    

In [31]:
valid_annot = pd.read_csv("data/01_raw/ValidationSet.csv")

valid_annot.sample(5)

valid_docx_folder = "data/01_raw/Validation_Data"

valid_fnames = os.listdir(valid_docx_folder)

clean_vfnames = [clean_train_names(x) for x in valid_fnames]

valid_annot_available = valid_annot.loc[valid_annot["File Name"].isin(clean_vfnames), :]

annotated_df_valid = annotate_all_docs_valid(valid_fnames, valid_annot_available)

annotated_df_valid["annot_count"] = annotated_df_valid["annotations"].apply(lambda x: len(set(x)))

annotated_df_valid["annot_count"].value_counts()

1    198
2     26
3     25
4      3
Name: annot_count, dtype: int64

In [0]:
annotated_df_valid.to_pickle("data/02_processed/annotated_df_valid.pickle")

In [36]:
!git status

!git add data/*/*.pickle

!git status


On branch master
Your branch is ahead of 'origin/master' by 1 commit.
  (use "git push" to publish your local commits)

Untracked files:
  (use "git add <file>..." to include in what will be committed)

	data/02_processed/annotated_df_v1.pickle
	data/02_processed/annotated_df_valid.pickle

nothing added to commit but untracked files present (use "git add" to track)
On branch master
Your branch is ahead of 'origin/master' by 1 commit.
  (use "git push" to publish your local commits)

Changes to be committed:
  (use "git reset HEAD <file>..." to unstage)

	new file:   data/02_processed/annotated_df_v1.pickle
	new file:   data/02_processed/annotated_df_valid.pickle



In [37]:
!git config --local user.email "narahari.bm@protonmail.com"
!git config --local user.name "Narahari B M"

!git commit -m "added annotated df for validation sets"

[master 0c0ceac] added annotated df for validation sets
 2 files changed, 0 insertions(+), 0 deletions(-)
 create mode 100644 data/02_processed/annotated_df_v1.pickle
 create mode 100644 data/02_processed/annotated_df_valid.pickle
fatal: could not read Username for 'https://github.com': No such device or address


In [38]:
!git push origin master

Counting objects: 6, done.
Delta compression using up to 2 threads.
Compressing objects: 100% (6/6), done.
Writing objects: 100% (6/6), 293.03 KiB | 3.49 MiB/s, done.
Total 6 (delta 0), reused 0 (delta 0)
To https://github.com/selfishhari/rental_meta_extraction.git
   e411e5b..0c0ceac  master -> master
